# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv('Orders.csv')
orders.shape
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

# "Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here
# 1. Aggregate amount_spent for unique customers.
customer_spending = orders.groupby('CustomerID').agg({'amount_spent':'sum'}).reset_index()
display(customer_spending.max())


# 2. Select customers whose aggregated amount_spent is in a given quantile range
# create bins for the customer amount_spent
Qzero = 0
Q1 = customer_spending['amount_spent'].quantile(0.25)
Q2 = customer_spending['amount_spent'].quantile(0.5) - Q1
Q3 = customer_spending['amount_spent'].quantile(0.95) - customer_spending['amount_spent'].quantile(0.75)
Q4 = customer_spending['amount_spent'].max()

# 3. Preferred customers will be those customers between the 75 and 95 percentile and VIP customers will be those above the 95 percentile
bins = ['Q1','Q2','Preferred','VIP']
customer_spending['customer_bins'] = pd.cut(customer_spending['amount_spent'],[Qzero,Q1,Q2,Q3,Q4], labels = bins)
customer_spending.sort_values(by = ['customer_bins'], ascending =False)
customer_bin_table = customer_spending.pivot_table(index = ['customer_bins','CustomerID'], values = ['amount_spent']).reset_index()
customer_bin_table



CustomerID       18287.00
amount_spent    280206.02
dtype: float64

,customer_bins,CustomerID,amount_spent
0,Q1,12353,89.00
1,Q1,12361,189.90
2,Q1,12367,168.90
3,Q1,12401,84.30
4,Q1,12402,225.60
...,...,...,...
4333,VIP,18223,6484.54
4334,VIP,18225,5509.12
4335,VIP,18226,5228.05
4336,VIP,18229,7276.90


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [8]:
# your code here
# merge the main table and the binned table
merged_tables = pd.merge(left = customer_spending,
                         right = orders,
                         how  = 'inner',
                         left_on = 'CustomerID',
                         right_on = 'CustomerID')

# allocate relevant columns to new dataframe
country_VIPs = merged_tables[['CustomerID','Country', 'customer_bins']]

# group rows by Country based on the value counts of each customer bin
country_VIPs = country_VIPs.groupby(['Country']).agg({'customer_bins':'value_counts'}).reset_index()
country_VIPs.rename(columns = {'level_1':'Customer Class'}, inplace = True)
country_VIPs


# sort values in descending order and select the first entry.
country_by_bin = country_VIPs.sort_values(by =['Customer Class','customer_bins'], ascending = False )
country_by_bin.head(10)


,Country,Customer Class,customer_bins
141,United Kingdom,VIP,115465
40,EIRE,VIP,7238
52,France,VIP,4487
57,Germany,VIP,4266
92,Netherlands,VIP,2080
121,Spain,VIP,1130
0,Australia,VIP,898
129,Switzerland,VIP,809
96,Norway,VIP,758
105,Portugal,VIP,681


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [12]:
# your code here

# group rows by Country based on the value counts of each customer bin
country_VIPs_Preferred = country_VIPs
country_VIPs_Preferred.rename(columns = {'level_1':'Customer Class'}, inplace = True)

# apply Boolean Mask to drop rows not in VIP or Preferred Bins.
country_VIP_Preferred = country_VIPs_Preferred[(country_VIPs_Preferred['Customer Class'] == 'VIP') | (country_VIPs_Preferred['Customer Class'] == 'Preferred')]

# create new dataframe from retained columns and group by the aggregated sum of VIP and Preffered customer_bins
country_VIP_Preferred = country_VIP_Preferred.groupby(['Country']).agg({'customer_bins':'sum'}).reset_index()

# sort values in descending order
country_VIP_Preferred.sort_values(by = 'customer_bins', ascending = False, inplace = True)

# select the top value
country_VIP_Preferred.head()


,Country,customer_bins
35,United Kingdom,329391
14,Germany,8790
13,France,8058
10,EIRE,7238
30,Spain,2417
